# 📡 ESP32 Access Point Implementation for Bengali Sign Language Recognition

This notebook implements fetching sensor data from ESP32 using Access Point mode instead of WebSocket, and uses the trained LSTM model for real-time Bengali character recognition.

## 🎯 Features:
- 📡 ESP32 Access Point HTTP communication
- 🤖 LSTM Bengali Sign Language recognition  
- 🔄 Real-time continuous prediction
- 📊 Live monitoring and statistics
- 🛡️ Connection resilience and error handling

In [39]:
# 📦 Import Required Libraries and Load LSTM Model
import numpy as np
import pandas as pd
import time
import threading
import requests
from collections import deque
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import json

# TensorFlow/Keras imports
try:
    import tensorflow as tf
    from tensorflow.keras.models import load_model
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    import joblib
    print("✅ All required libraries imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("📦 Installing required packages...")
    !pip install tensorflow scikit-learn requests matplotlib

# Load the trained LSTM model and preprocessing components
def load_bengali_lstm_model():
    """Load the trained LSTM model and preprocessing components"""
    
    print("🔄 Loading Bengali Sign Language LSTM Model...")
    
    try:
        # Model paths
        model_path = '/home/meraj/Downloads/BdSL_Bangla Sign_Language/sarborno_lstm_model.h5'
        preprocessing_path = '/home/meraj/Downloads/BdSL_Bangla Sign_Language/sarborno_lstm_preprocessing.pkl'
        
        # Load LSTM model
        print("📂 Loading LSTM model...")
        lstm_model = load_model(model_path)
        print(f"✅ LSTM model loaded: {lstm_model.input_shape}")
        
        # Load preprocessing components
        print("📂 Loading preprocessing components...")
        preprocessing_components = joblib.load(preprocessing_path)
        
        scaler = preprocessing_components['scaler']
        label_encoder = preprocessing_components['label_encoder']
        
        print(f"✅ Scaler loaded: {scaler.__class__.__name__}")
        print(f"✅ Label encoder loaded with {len(label_encoder.classes_)} classes")
        print(f"📝 Characters: {label_encoder.classes_[:10]}... (showing first 10)")
        
        return lstm_model, scaler, label_encoder
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        print("💡 Make sure the model files exist:")
        print(f"   • {model_path}")
        print(f"   • {preprocessing_path}")
        return None, None, None

# Load the model
print("🚀 INITIALIZING BENGALI SIGN LANGUAGE RECOGNITION SYSTEM...")
print("=" * 60)

lstm_model, scaler, label_encoder = load_bengali_lstm_model()

if lstm_model is not None:
    print("\n🎉 MODEL LOADING SUCCESSFUL!")
    print("✅ Ready for Bengali Sign Language recognition")
    
    # Display model summary
    print(f"\n📊 MODEL INFORMATION:")
    print(f"   • Input shape: {lstm_model.input_shape}")
    print(f"   • Output classes: {len(label_encoder.classes_)}")
    print(f"   • Model layers: {len(lstm_model.layers)}")
    
else:
    print("\n❌ MODEL LOADING FAILED!")
    print("💡 Please ensure the model files are available")

✅ All required libraries imported successfully!
🚀 INITIALIZING BENGALI SIGN LANGUAGE RECOGNITION SYSTEM...
🔄 Loading Bengali Sign Language LSTM Model...
📂 Loading LSTM model...
✅ LSTM model loaded: (None, 1, 10)
📂 Loading preprocessing components...
✅ Scaler loaded: StandardScaler
✅ Label encoder loaded with 9 classes
📝 Characters: ['অ' 'আ' 'ই' 'উ' 'ঋ' 'এ' 'ঐ' 'ও' 'ঔ']... (showing first 10)

🎉 MODEL LOADING SUCCESSFUL!
✅ Ready for Bengali Sign Language recognition

📊 MODEL INFORMATION:
   • Input shape: (None, 1, 10)
   • Output classes: 9
   • Model layers: 12


In [40]:
# 🌐 ESP32 Access Point HTTP Client
class ESP32AccessPointClient:
    """
    HTTP client for connecting to ESP32 in Access Point mode
    Provides stable HTTP-based communication as alternative to WebSocket
    """
    
    def __init__(self, esp32_ip="192.168.10.88", port=80, timeout=5):
        """
        Initialize ESP32 Access Point client
        
        Args:
            esp32_ip: IP address of ESP32 Access Point (default: 192.168.10.88)
            port: HTTP port (default: 80)
            timeout: Request timeout in seconds
        """
        self.esp32_ip = esp32_ip
        self.port = port
        self.base_url = f"http://{esp32_ip}:{port}"
        self.timeout = timeout
        self.session = requests.Session()
        
        # Connection statistics
        self.stats = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'connection_errors': 0,
            'timeout_errors': 0,
            'last_success_time': None,
            'last_error_time': None
        }
        
        print(f"🌐 ESP32AccessPointClient initialized")
        print(f"   📡 Base URL: {self.base_url}")
        print(f"   ⏱️ Timeout: {timeout}s")
    
    def test_connection(self):
        """Test connection to ESP32 Access Point"""
        print(f"🔍 Testing connection to ESP32 at {self.base_url}...")
        
        try:
            response = self.session.get(f"{self.base_url}/status", timeout=self.timeout)
            if response.status_code == 200:
                print("✅ ESP32 Access Point connection successful!")
                return True
            else:
                print(f"⚠️ ESP32 responded with status: {response.status_code}")
                return False
                
        except requests.exceptions.ConnectionError:
            print("❌ Connection error: Cannot reach ESP32")
            return False
        except requests.exceptions.Timeout:
            print("❌ Timeout error: ESP32 not responding")
            return False
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            return False
    
    def get_sensor_data(self, endpoint="/data"):
        """
        Fetch sensor data from ESP32 via HTTP GET request
        
        Args:
            endpoint: API endpoint to fetch data from
            
        Returns:
            dict: Sensor data or None if failed
        """
        url = f"{self.base_url}{endpoint}"
        
        try:
            self.stats['total_requests'] += 1
            
            response = self.session.get(url, timeout=self.timeout)
            
            if response.status_code == 200:
                self.stats['successful_requests'] += 1
                self.stats['last_success_time'] = time.time()
                
                # Try to parse as JSON
                try:
                    data = response.json()
                    return data
                except json.JSONDecodeError:
                    # If not JSON, try to parse as CSV-like data
                    text_data = response.text.strip()
                    if text_data:
                        return self._parse_csv_data(text_data)
                    return None
                    
            else:
                self.stats['failed_requests'] += 1
                print(f"⚠️ HTTP {response.status_code}: {response.text[:100]}")
                return None
                
        except requests.exceptions.ConnectionError:
            self.stats['connection_errors'] += 1
            self.stats['last_error_time'] = time.time()
            return None
            
        except requests.exceptions.Timeout:
            self.stats['timeout_errors'] += 1
            self.stats['last_error_time'] = time.time()
            return None
            
        except Exception as e:
            self.stats['failed_requests'] += 1
            self.stats['last_error_time'] = time.time()
            print(f"❌ Error fetching data: {e}")
            return None
    
    def _parse_csv_data(self, csv_text):
        """
        Parse CSV-formatted sensor data from ESP32
        
        Args:
            csv_text: Raw CSV text data
            
        Returns:
            dict: Parsed sensor data
        """
        try:
            # Split by lines and take the last non-empty line
            lines = [line.strip() for line in csv_text.split('\n') if line.strip()]
            if not lines:
                return None
            
            # Take the most recent data (last line)
            data_line = lines[-1]
            
            # Split by comma and convert to float
            values = [float(x.strip()) for x in data_line.split(',')]
            
            # Create structured data (assuming 6 sensor values)
            if len(values) >= 6:
                return {
                    'timestamp': time.time(),
                    'sensor_data': values,
                    'sensor_count': len(values)
                }
            else:
                print(f"⚠️ Incomplete data: expected 6+ values, got {len(values)}")
                return None
                
        except Exception as e:
            print(f"❌ Error parsing CSV data: {e}")
            return None
    
    def get_connection_stats(self):
        """Get connection statistics"""
        success_rate = 0
        if self.stats['total_requests'] > 0:
            success_rate = (self.stats['successful_requests'] / self.stats['total_requests']) * 100
        
        return {
            **self.stats,
            'success_rate': success_rate,
            'connection_reliability': 'High' if success_rate >= 90 else 'Medium' if success_rate >= 70 else 'Low'
        }
    
    def print_stats(self):
        """Print connection statistics"""
        stats = self.get_connection_stats()
        
        print(f"\n📊 ESP32 ACCESS POINT CONNECTION STATISTICS:")
        print(f"   📈 Total requests: {stats['total_requests']}")
        print(f"   ✅ Successful: {stats['successful_requests']}")
        print(f"   ❌ Failed: {stats['failed_requests']}")
        print(f"   🔗 Connection errors: {stats['connection_errors']}")
        print(f"   ⏱️ Timeout errors: {stats['timeout_errors']}")
        print(f"   📊 Success rate: {stats['success_rate']:.1f}%")
        print(f"   🎯 Reliability: {stats['connection_reliability']}")
        
        if stats['last_success_time']:
            last_success = time.time() - stats['last_success_time']
            print(f"   🕒 Last success: {last_success:.1f}s ago")

# Initialize ESP32 Access Point Client
print("🚀 INITIALIZING ESP32 ACCESS POINT CLIENT...")
print("=" * 50)

esp32_client = ESP32AccessPointClient(esp32_ip="192.168.4.1", port=80, timeout=5)

# Test connection
if esp32_client.test_connection():
    print("🎉 ESP32 Access Point ready for data communication!")
else:
    print("⚠️ ESP32 Access Point connection failed - will retry during operation")

🚀 INITIALIZING ESP32 ACCESS POINT CLIENT...
🌐 ESP32AccessPointClient initialized
   📡 Base URL: http://192.168.4.1:80
   ⏱️ Timeout: 5s
🔍 Testing connection to ESP32 at http://192.168.4.1:80...
⚠️ ESP32 responded with status: 404
⚠️ ESP32 Access Point connection failed - will retry during operation


In [41]:
# 🧠 Bengali Sign Language LSTM Predictor for Access Point
class AccessPointBengaliPredictor:
    """
    Bengali Sign Language predictor using LSTM model with ESP32 Access Point data
    Provides real-time prediction capabilities with HTTP-based communication
    """
    
    def __init__(self, lstm_model, scaler, label_encoder, esp32_client):
        """
        Initialize the predictor
        
        Args:
            lstm_model: Trained LSTM model
            scaler: Data scaler for preprocessing
            label_encoder: Label encoder for character mapping
            esp32_client: ESP32AccessPointClient instance
        """
        self.lstm_model = lstm_model
        self.scaler = scaler
        self.label_encoder = label_encoder
        self.esp32_client = esp32_client
        
        # Get model input requirements
        self.input_shape = lstm_model.input_shape
        self.sequence_length = self.input_shape[1]  # Time steps
        self.feature_count = self.input_shape[2]    # Features per time step
        
        # Data buffer for sequence building
        self.data_buffer = deque(maxlen=self.sequence_length)
        
        # Prediction statistics
        self.prediction_stats = {
            'total_predictions': 0,
            'successful_predictions': 0,
            'confident_predictions': 0,
            'data_fetch_attempts': 0,
            'data_fetch_successes': 0,
            'last_prediction_time': None,
            'last_character': None,
            'last_confidence': None
        }
        
        print(f"🧠 AccessPointBengaliPredictor initialized")
        print(f"   📊 Model input shape: {self.input_shape}")
        print(f"   📏 Sequence length: {self.sequence_length}")
        print(f"   🎯 Features per step: {self.feature_count}")
        print(f"   🔤 Character classes: {len(self.label_encoder.classes_)}")
    
    def fetch_and_predict(self, confidence_threshold=0.8):
        """
        Fetch data from ESP32 and make prediction
        
        Args:
            confidence_threshold: Minimum confidence for valid prediction
            
        Returns:
            dict: Prediction result with character, confidence, and status
        """
        self.prediction_stats['data_fetch_attempts'] += 1
        
        # Fetch sensor data from ESP32
        sensor_data = self.esp32_client.get_sensor_data()
        
        if sensor_data is None:
            return {
                'success': False,
                'error': 'Failed to fetch data from ESP32',
                'character': None,
                'confidence': None,
                'timestamp': time.time()
            }
        
        self.prediction_stats['data_fetch_successes'] += 1
        
        # Extract sensor values
        if 'sensor_data' in sensor_data:
            sensor_values = sensor_data['sensor_data']
        else:
            return {
                'success': False,
                'error': 'Invalid sensor data format',
                'character': None,
                'confidence': None,
                'timestamp': time.time()
            }
        
        # Ensure we have enough features
        if len(sensor_values) < self.feature_count:
            return {
                'success': False,
                'error': f'Insufficient features: got {len(sensor_values)}, need {self.feature_count}',
                'character': None,
                'confidence': None,
                'timestamp': time.time()
            }
        
        # Take only the required number of features
        features = sensor_values[:self.feature_count]
        
        # Add to buffer
        self.data_buffer.append(features)
        
        # Check if we have enough data for prediction
        if len(self.data_buffer) < self.sequence_length:
            return {
                'success': False,
                'error': f'Building sequence: {len(self.data_buffer)}/{self.sequence_length}',
                'character': None,
                'confidence': None,
                'timestamp': time.time(),
                'buffer_progress': len(self.data_buffer) / self.sequence_length
            }
        
        # Make prediction
        try:
            # Prepare data for prediction
            sequence_data = np.array(list(self.data_buffer))
            sequence_data_scaled = self.scaler.transform(sequence_data)
            prediction_input = sequence_data_scaled.reshape(1, self.sequence_length, self.feature_count)
            
            # Get prediction
            prediction_probs = self.lstm_model.predict(prediction_input, verbose=0)
            predicted_class_idx = np.argmax(prediction_probs[0])
            confidence = float(prediction_probs[0][predicted_class_idx])
            
            # Decode character
            predicted_character = self.label_encoder.inverse_transform([predicted_class_idx])[0]
            
            # Update statistics
            self.prediction_stats['total_predictions'] += 1
            self.prediction_stats['last_prediction_time'] = time.time()
            self.prediction_stats['last_character'] = predicted_character
            self.prediction_stats['last_confidence'] = confidence
            
            if confidence >= confidence_threshold:
                self.prediction_stats['confident_predictions'] += 1
                self.prediction_stats['successful_predictions'] += 1
                
                return {
                    'success': True,
                    'character': predicted_character,
                    'confidence': confidence,
                    'timestamp': time.time(),
                    'is_confident': True,
                    'sensor_data': features
                }
            else:
                return {
                    'success': True,
                    'character': predicted_character,
                    'confidence': confidence,
                    'timestamp': time.time(),
                    'is_confident': False,
                    'warning': f'Low confidence: {confidence:.3f} < {confidence_threshold}',
                    'sensor_data': features
                }
                
        except Exception as e:
            return {
                'success': False,
                'error': f'Prediction error: {e}',
                'character': None,
                'confidence': None,
                'timestamp': time.time()
            }
    
    def get_prediction_stats(self):
        """Get prediction statistics"""
        stats = self.prediction_stats.copy()
        
        # Calculate rates
        if stats['data_fetch_attempts'] > 0:
            stats['data_fetch_rate'] = (stats['data_fetch_successes'] / stats['data_fetch_attempts']) * 100
        else:
            stats['data_fetch_rate'] = 0
        
        if stats['total_predictions'] > 0:
            stats['confidence_rate'] = (stats['confident_predictions'] / stats['total_predictions']) * 100
            stats['success_rate'] = (stats['successful_predictions'] / stats['total_predictions']) * 100
        else:
            stats['confidence_rate'] = 0
            stats['success_rate'] = 0
        
        return stats
    
    def print_stats(self):
        """Print prediction statistics"""
        stats = self.get_prediction_stats()
        
        print(f"\n🧠 BENGALI PREDICTOR STATISTICS:")
        print(f"   📡 Data fetch attempts: {stats['data_fetch_attempts']}")
        print(f"   ✅ Data fetch successes: {stats['data_fetch_successes']}")
        print(f"   📊 Data fetch rate: {stats['data_fetch_rate']:.1f}%")
        print(f"   🎯 Total predictions: {stats['total_predictions']}")
        print(f"   ✨ Confident predictions: {stats['confident_predictions']}")
        print(f"   📈 Confidence rate: {stats['confidence_rate']:.1f}%")
        print(f"   🎉 Success rate: {stats['success_rate']:.1f}%")
        
        if stats['last_character']:
            print(f"   🔤 Last character: {stats['last_character']}")
            print(f"   💪 Last confidence: {stats['last_confidence']:.3f}")

# Initialize Bengali Predictor (only if model is loaded)
if lstm_model is not None and scaler is not None and label_encoder is not None:
    print("🚀 INITIALIZING BENGALI PREDICTOR WITH ACCESS POINT...")
    print("=" * 55)
    
    bengali_predictor = AccessPointBengaliPredictor(
        lstm_model=lstm_model,
        scaler=scaler,
        label_encoder=label_encoder,
        esp32_client=esp32_client
    )
    
    print("🎉 Bengali predictor ready for Access Point communication!")
    
else:
    print("❌ Cannot initialize predictor - model components not loaded")
    bengali_predictor = None

🚀 INITIALIZING BENGALI PREDICTOR WITH ACCESS POINT...
🧠 AccessPointBengaliPredictor initialized
   📊 Model input shape: (None, 1, 10)
   📏 Sequence length: 1
   🎯 Features per step: 10
   🔤 Character classes: 9
🎉 Bengali predictor ready for Access Point communication!


In [42]:
# 🎯 Single Prediction Test
def test_single_prediction(show_details=True):
    """Test a single prediction from ESP32 Access Point"""
    
    if bengali_predictor is None:
        print("❌ Predictor not available")
        return
    
    print("🎯 TESTING SINGLE PREDICTION FROM ESP32 ACCESS POINT...")
    print("=" * 55)
    
    result = bengali_predictor.fetch_and_predict(confidence_threshold=0.7)
    
    if show_details:
        print(f"⏰ Timestamp: {time.strftime('%H:%M:%S', time.localtime(result['timestamp']))}")
        
        if result['success']:
            if result.get('is_confident', False):
                print(f"🎉 SUCCESS: Predicted '{result['character']}' with {result['confidence']:.1%} confidence")
            else:
                print(f"⚠️ LOW CONFIDENCE: '{result['character']}' ({result['confidence']:.1%})")
                if 'warning' in result:
                    print(f"   💡 {result['warning']}")
        else:
            print(f"❌ FAILED: {result['error']}")
            if 'buffer_progress' in result:
                progress = result['buffer_progress'] * 100
                print(f"   📊 Buffer progress: {progress:.1f}%")
        
        # Show sensor data if available
        if 'sensor_data' in result:
            sensor_summary = [f"{x:.2f}" for x in result['sensor_data'][:3]]
            print(f"   📡 Sensor data: [{', '.join(sensor_summary)}...]")
    
    return result

# Test the prediction
if bengali_predictor is not None:
    print("🔍 Running single prediction test...")
    test_result = test_single_prediction()
else:
    print("⚠️ Cannot test - predictor not initialized")

🔍 Running single prediction test...
🎯 TESTING SINGLE PREDICTION FROM ESP32 ACCESS POINT...
⚠️ HTTP 404: Not found
⏰ Timestamp: 00:46:03
❌ FAILED: Failed to fetch data from ESP32


In [43]:
# 🔄 Continuous Real-Time Prediction with Access Point
class ContinuousAccessPointPredictor:
    """
    Continuous prediction system using ESP32 Access Point
    Provides real-time Bengali Sign Language recognition with HTTP polling
    """
    
    def __init__(self, bengali_predictor, polling_interval=0.2):
        """
        Initialize continuous predictor
        
        Args:
            bengali_predictor: AccessPointBengaliPredictor instance
            polling_interval: Time between HTTP requests (seconds)
        """
        self.bengali_predictor = bengali_predictor
        self.polling_interval = polling_interval
        self.is_running = False
        self.thread = None
        
        # Results tracking
        self.results_history = deque(maxlen=100)
        self.session_stats = {
            'start_time': None,
            'total_polls': 0,
            'successful_predictions': 0,
            'confident_predictions': 0,
            'http_errors': 0,
            'prediction_errors': 0,
            'characters_detected': set(),
            'latest_character': None,
            'latest_confidence': None
        }
        
        print(f"🔄 ContinuousAccessPointPredictor initialized")
        print(f"   ⏱️ Polling interval: {polling_interval}s ({1/polling_interval:.1f} Hz)")
    
    def start_continuous_prediction(self, duration_seconds=30, confidence_threshold=0.8):
        """
        Start continuous prediction for specified duration
        
        Args:
            duration_seconds: How long to run prediction (seconds)
            confidence_threshold: Minimum confidence for valid predictions
        """
        if self.is_running:
            print("⚠️ Continuous prediction already running!")
            return
        
        print(f"🚀 STARTING CONTINUOUS PREDICTION...")
        print(f"   ⏰ Duration: {duration_seconds}s")
        print(f"   🎯 Confidence threshold: {confidence_threshold}")
        print(f"   📡 Polling rate: {1/self.polling_interval:.1f} Hz")
        print("   🛑 Press Ctrl+C to stop early")
        print("=" * 50)
        
        self.is_running = True
        self.session_stats['start_time'] = time.time()
        
        def prediction_loop():
            try:
                end_time = time.time() + duration_seconds
                
                while self.is_running and time.time() < end_time:
                    loop_start = time.time()
                    
                    # Make prediction
                    self.session_stats['total_polls'] += 1
                    result = self.bengali_predictor.fetch_and_predict(confidence_threshold)
                    
                    # Process result
                    self._process_prediction_result(result, confidence_threshold)
                    
                    # Store in history
                    self.results_history.append({
                        'timestamp': time.time(),
                        'result': result,
                        'poll_number': self.session_stats['total_polls']
                    })
                    
                    # Display result
                    self._display_prediction_result(result)
                    
                    # Maintain polling rate
                    elapsed = time.time() - loop_start
                    sleep_time = max(0, self.polling_interval - elapsed)
                    if sleep_time > 0:
                        time.sleep(sleep_time)
                
                print("\\n✅ Continuous prediction completed!")
                
            except KeyboardInterrupt:
                print("\\n🛑 Prediction stopped by user")
            except Exception as e:
                print(f"\\n❌ Error in prediction loop: {e}")
            finally:
                self.is_running = False
        
        # Start prediction in thread
        self.thread = threading.Thread(target=prediction_loop, daemon=True)
        self.thread.start()
        
        return self.thread
    
    def _process_prediction_result(self, result, confidence_threshold):
        """Process and categorize prediction result"""
        if result['success']:
            self.session_stats['successful_predictions'] += 1
            
            if result.get('is_confident', False):
                self.session_stats['confident_predictions'] += 1
                self.session_stats['characters_detected'].add(result['character'])
                self.session_stats['latest_character'] = result['character']
                self.session_stats['latest_confidence'] = result['confidence']
            
        else:
            if 'Failed to fetch data' in result.get('error', ''):
                self.session_stats['http_errors'] += 1
            else:
                self.session_stats['prediction_errors'] += 1
    
    def _display_prediction_result(self, result):
        """Display prediction result in real-time"""
        timestamp = time.strftime('%H:%M:%S', time.localtime(result['timestamp']))
        poll_num = self.session_stats['total_polls']
        
        if result['success']:
            char = result['character']
            conf = result['confidence']
            
            if result.get('is_confident', False):
                print(f"[{timestamp}] Poll #{poll_num:3d}: ✅ '{char}' ({conf:.1%})")
            else:
                print(f"[{timestamp}] Poll #{poll_num:3d}: ⚠️ '{char}' ({conf:.1%}) - Low confidence")
        else:
            error = result['error']
            if 'Building sequence' in error:
                progress = result.get('buffer_progress', 0) * 100
                print(f"[{timestamp}] Poll #{poll_num:3d}: 📊 Building buffer... {progress:.0f}%")
            else:
                print(f"[{timestamp}] Poll #{poll_num:3d}: ❌ {error}")
    
    def stop_prediction(self):
        """Stop continuous prediction"""
        if self.is_running:
            self.is_running = False
            print("🛑 Stopping continuous prediction...")
            if self.thread:
                self.thread.join(timeout=2)
            print("✅ Prediction stopped")
        else:
            print("ℹ️ Prediction not running")
    
    def get_session_summary(self):
        """Get summary of current prediction session"""
        if self.session_stats['start_time'] is None:
            return None
        
        elapsed = time.time() - self.session_stats['start_time']
        stats = self.session_stats.copy()
        
        # Calculate rates
        if stats['total_polls'] > 0:
            stats['success_rate'] = (stats['successful_predictions'] / stats['total_polls']) * 100
            stats['confidence_rate'] = (stats['confident_predictions'] / stats['total_polls']) * 100
            stats['polling_rate'] = stats['total_polls'] / elapsed if elapsed > 0 else 0
        else:
            stats['success_rate'] = 0
            stats['confidence_rate'] = 0
            stats['polling_rate'] = 0
        
        stats['elapsed_time'] = elapsed
        stats['characters_detected'] = list(stats['characters_detected'])
        
        return stats
    
    def print_session_summary(self):
        """Print detailed session summary"""
        summary = self.get_session_summary()
        
        if summary is None:
            print("📊 No session data available")
            return
        
        print(f"\\n📊 CONTINUOUS PREDICTION SESSION SUMMARY:")
        print(f"   ⏰ Duration: {summary['elapsed_time']:.1f}s")
        print(f"   📡 Total polls: {summary['total_polls']}")
        print(f"   📈 Polling rate: {summary['polling_rate']:.1f} Hz")
        print(f"   ✅ Successful predictions: {summary['successful_predictions']}")
        print(f"   💪 Confident predictions: {summary['confident_predictions']}")
        print(f"   🌐 HTTP errors: {summary['http_errors']}")
        print(f"   🤖 Prediction errors: {summary['prediction_errors']}")
        print(f"   📊 Success rate: {summary['success_rate']:.1f}%")
        print(f"   🎯 Confidence rate: {summary['confidence_rate']:.1f}%")
        
        if summary['characters_detected']:
            print(f"   🔤 Characters detected: {', '.join(summary['characters_detected'])}")
        
        if summary['latest_character']:
            print(f"   🏆 Latest prediction: '{summary['latest_character']}' ({summary['latest_confidence']:.1%})")

# Initialize continuous predictor
if bengali_predictor is not None:
    print("🚀 INITIALIZING CONTINUOUS ACCESS POINT PREDICTOR...")
    print("=" * 55)
    
    continuous_predictor = ContinuousAccessPointPredictor(
        bengali_predictor=bengali_predictor,
        polling_interval=0.2  # 5 Hz polling rate
    )
    
    print("🎉 Continuous predictor ready!")
    print("💡 Use: continuous_predictor.start_continuous_prediction(duration_seconds=30)")
    
else:
    print("❌ Cannot initialize continuous predictor - Bengali predictor not available")
    continuous_predictor = None

🚀 INITIALIZING CONTINUOUS ACCESS POINT PREDICTOR...
🔄 ContinuousAccessPointPredictor initialized
   ⏱️ Polling interval: 0.2s (5.0 Hz)
🎉 Continuous predictor ready!
💡 Use: continuous_predictor.start_continuous_prediction(duration_seconds=30)


In [44]:
# 🚀 Quick Start Demo - ESP32 Access Point Bengali Recognition
def demo_access_point_recognition(duration=20):
    """
    Quick demonstration of ESP32 Access Point Bengali Sign Language recognition
    
    Args:
        duration: Demo duration in seconds
    """
    print("🚀 ESP32 ACCESS POINT BENGALI RECOGNITION DEMO")
    print("=" * 50)
    
    if continuous_predictor is None:
        print("❌ Continuous predictor not available")
        return
    
    # Check ESP32 connection
    print("🔍 Checking ESP32 Access Point connection...")
    if not esp32_client.test_connection():
        print("⚠️ ESP32 connection failed, but continuing with demo...")
        print("💡 Make sure ESP32 is in Access Point mode at 192.168.10.88")
    
    print(f"\\n🎬 Starting {duration}s demo...")
    print("📱 Move your glove to perform Bengali sign gestures")
    print("🎯 System will recognize characters with HTTP polling")
    print("🛑 Press Ctrl+C to stop early")
    
    # Start continuous prediction
    thread = continuous_predictor.start_continuous_prediction(
        duration_seconds=duration,
        confidence_threshold=0.75
    )
    
    # Wait for completion
    if thread:
        try:
            thread.join()
        except KeyboardInterrupt:
            continuous_predictor.stop_prediction()
    
    # Show final statistics
    print("\\n" + "=" * 50)
    continuous_predictor.print_session_summary()
    
    # Show connection stats
    esp32_client.print_stats()
    
    # Show predictor stats
    bengali_predictor.print_stats()

print("🎉 ACCESS POINT IMPLEMENTATION READY!")
print("=" * 40)
print("💡 Available functions:")
print("   • test_single_prediction() - Test one prediction")
print("   • continuous_predictor.start_continuous_prediction(30) - 30s continuous")
print("   • demo_access_point_recognition(20) - 20s complete demo")
print("   • esp32_client.print_stats() - Connection statistics")
print("   • bengali_predictor.print_stats() - Prediction statistics")
print()
print("🚀 Quick start: demo_access_point_recognition(20)")

🎉 ACCESS POINT IMPLEMENTATION READY!
💡 Available functions:
   • test_single_prediction() - Test one prediction
   • continuous_predictor.start_continuous_prediction(30) - 30s continuous
   • demo_access_point_recognition(20) - 20s complete demo
   • esp32_client.print_stats() - Connection statistics
   • bengali_predictor.print_stats() - Prediction statistics

🚀 Quick start: demo_access_point_recognition(20)


In [25]:
# This entire pipeline works offline:
def offline_bengali_recognition():
    """Complete offline Bengali sign recognition"""
    
    # 1. ESP32 provides local WiFi (no internet)
    esp32_client = ESP32AccessPointClient("192.168.4.1")
    
    # 2. Get sensor data via local WiFi
    sensor_data = esp32_client.get_sensor_data()
    
    # 3. AI prediction happens locally
    result = bengali_predictor.fetch_and_predict()
    
    # 4. Bengali character recognized offline!
    return result['character']  # e.g., 'অ', 'আ', 'ই'

In [45]:
# 📈 Enhanced Continuous Data Prediction with Real-Time Monitoring
class EnhancedContinuousPredictor:
    """
    Advanced continuous prediction system with real-time data visualization
    and comprehensive monitoring capabilities
    """
    
    def __init__(self, bengali_predictor, polling_interval=0.2):
        """
        Initialize enhanced continuous predictor
        
        Args:
            bengali_predictor: AccessPointBengaliPredictor instance
            polling_interval: Time between predictions (seconds)
        """
        self.bengali_predictor = bengali_predictor
        self.polling_interval = polling_interval
        self.is_running = False
        self.thread = None
        
        # Enhanced data tracking
        self.prediction_history = deque(maxlen=200)  # Store more history
        self.confidence_history = deque(maxlen=200)
        self.sensor_data_history = deque(maxlen=200)
        self.timestamp_history = deque(maxlen=200)
        
        # Real-time statistics
        self.live_stats = {
            'start_time': None,
            'total_predictions': 0,
            'successful_predictions': 0,
            'confident_predictions': 0,
            'average_confidence': 0.0,
            'prediction_rate': 0.0,
            'characters_this_session': {},
            'current_character': None,
            'current_confidence': 0.0,
            'last_update_time': None,
            'data_quality_score': 0.0
        }
        
        # Performance metrics
        self.performance_metrics = {
            'prediction_times': deque(maxlen=50),
            'network_latencies': deque(maxlen=50),
            'cpu_usage_samples': deque(maxlen=50),
            'memory_usage_samples': deque(maxlen=50)
        }
        
        print(f"📈 EnhancedContinuousPredictor initialized")
        print(f"   ⚡ Polling rate: {1/polling_interval:.1f} Hz")
        print(f"   💾 History buffer: {self.prediction_history.maxlen} samples")
    
    def start_enhanced_prediction(self, duration_seconds=60, confidence_threshold=0.8, 
                                show_live_stats=True, update_interval=2.0):
        """
        Start enhanced continuous prediction with live monitoring
        
        Args:
            duration_seconds: Prediction duration
            confidence_threshold: Minimum confidence for valid predictions
            show_live_stats: Whether to show live statistics updates
            update_interval: How often to update live stats (seconds)
        """
        if self.is_running:
            print("⚠️ Enhanced prediction already running!")
            return
        
        print("🚀 STARTING ENHANCED CONTINUOUS PREDICTION...")
        print(f"   ⏰ Duration: {duration_seconds}s")
        print(f"   🎯 Confidence threshold: {confidence_threshold:.1%}")
        print(f"   📡 Polling rate: {1/self.polling_interval:.1f} Hz")
        print(f"   📊 Live stats: {'Enabled' if show_live_stats else 'Disabled'}")
        print("=" * 60)
        
        self.is_running = True
        self.live_stats['start_time'] = time.time()
        last_stats_update = time.time()
        
        def enhanced_prediction_loop():
            try:
                end_time = time.time() + duration_seconds
                
                while self.is_running and time.time() < end_time:
                    loop_start = time.time()
                    
                    # Make prediction with timing
                    prediction_start = time.time()
                    result = self.bengali_predictor.fetch_and_predict(confidence_threshold)
                    prediction_time = time.time() - prediction_start
                    
                    # Store performance data
                    self.performance_metrics['prediction_times'].append(prediction_time)
                    
                    # Process and store result
                    self._process_enhanced_result(result, confidence_threshold)
                    
                    # Update live statistics
                    current_time = time.time()
                    if show_live_stats and (current_time - last_stats_update) >= update_interval:
                        self._update_live_stats()
                        self._display_live_dashboard()
                        last_stats_update = current_time
                    
                    # Maintain polling rate
                    elapsed = time.time() - loop_start
                    sleep_time = max(0, self.polling_interval - elapsed)
                    if sleep_time > 0:
                        time.sleep(sleep_time)
                
                print("\\n✅ Enhanced continuous prediction completed!")
                self._display_final_summary()
                
            except KeyboardInterrupt:
                print("\\n🛑 Enhanced prediction stopped by user")
                self._display_final_summary()
            except Exception as e:
                print(f"\\n❌ Error in enhanced prediction: {e}")
            finally:
                self.is_running = False
        
        # Start in separate thread
        self.thread = threading.Thread(target=enhanced_prediction_loop, daemon=True)
        self.thread.start()
        
        return self.thread
    
    def _process_enhanced_result(self, result, confidence_threshold):
        """Process prediction result with enhanced data tracking"""
        timestamp = time.time()
        self.timestamp_history.append(timestamp)
        self.live_stats['total_predictions'] += 1
        self.live_stats['last_update_time'] = timestamp
        
        if result['success']:
            character = result['character']
            confidence = result['confidence']
            
            # Store prediction data
            self.prediction_history.append(character)
            self.confidence_history.append(confidence)
            
            # Store sensor data if available
            if 'sensor_data' in result:
                self.sensor_data_history.append(result['sensor_data'])
            
            # Update character frequency
            if character not in self.live_stats['characters_this_session']:
                self.live_stats['characters_this_session'][character] = 0
            self.live_stats['characters_this_session'][character] += 1
            
            # Update current state
            self.live_stats['current_character'] = character
            self.live_stats['current_confidence'] = confidence
            self.live_stats['successful_predictions'] += 1
            
            if confidence >= confidence_threshold:
                self.live_stats['confident_predictions'] += 1
    
    def _update_live_stats(self):
        """Update live statistics"""
        if self.live_stats['total_predictions'] > 0:
            # Calculate average confidence
            if self.confidence_history:
                self.live_stats['average_confidence'] = np.mean(list(self.confidence_history))
            
            # Calculate prediction rate
            elapsed = time.time() - self.live_stats['start_time']
            if elapsed > 0:
                self.live_stats['prediction_rate'] = self.live_stats['total_predictions'] / elapsed
            
            # Calculate data quality score (based on success rate and confidence)
            success_rate = self.live_stats['successful_predictions'] / self.live_stats['total_predictions']
            confidence_rate = self.live_stats['confident_predictions'] / self.live_stats['total_predictions']
            self.live_stats['data_quality_score'] = (success_rate + confidence_rate) / 2
    
    def _display_live_dashboard(self):
        """Display real-time dashboard"""
        clear_output(wait=True)
        
        print("📊 LIVE BENGALI SIGN RECOGNITION DASHBOARD")
        print("=" * 60)
        
        # Current prediction
        current_char = self.live_stats['current_character'] or 'None'
        current_conf = self.live_stats['current_confidence']
        print(f"🎯 CURRENT: '{current_char}' ({current_conf:.1%} confidence)")
        
        # Statistics
        print(f"\\n📈 LIVE STATISTICS:")
        print(f"   ⏱️ Running time: {time.time() - self.live_stats['start_time']:.1f}s")
        print(f"   📡 Total predictions: {self.live_stats['total_predictions']}")
        print(f"   ✅ Successful: {self.live_stats['successful_predictions']}")
        print(f"   💪 Confident: {self.live_stats['confident_predictions']}")
        print(f"   📊 Avg confidence: {self.live_stats['average_confidence']:.1%}")
        print(f"   🚀 Prediction rate: {self.live_stats['prediction_rate']:.1f} Hz")
        print(f"   🎯 Data quality: {self.live_stats['data_quality_score']:.1%}")
        
        # Character frequency
        if self.live_stats['characters_this_session']:
            print(f"\\n🔤 CHARACTERS DETECTED:")
            sorted_chars = sorted(self.live_stats['characters_this_session'].items(), 
                                key=lambda x: x[1], reverse=True)
            for char, count in sorted_chars[:5]:  # Show top 5
                percentage = (count / self.live_stats['successful_predictions']) * 100
                print(f"   '{char}': {count} times ({percentage:.1f}%)")
        
        # Performance metrics
        if self.performance_metrics['prediction_times']:
            avg_pred_time = np.mean(list(self.performance_metrics['prediction_times']))
            print(f"\\n⚡ PERFORMANCE:")
            print(f"   🔄 Avg prediction time: {avg_pred_time:.3f}s")
        
        print("\\n" + "=" * 60)
    
    def _display_final_summary(self):
        """Display comprehensive final summary"""
        total_time = time.time() - self.live_stats['start_time']
        
        print("\\n📋 ENHANCED PREDICTION SESSION SUMMARY")
        print("=" * 50)
        
        # Basic statistics
        print(f"⏰ Total time: {total_time:.1f}s")
        print(f"📡 Total predictions: {self.live_stats['total_predictions']}")
        print(f"✅ Successful predictions: {self.live_stats['successful_predictions']}")
        print(f"💪 Confident predictions: {self.live_stats['confident_predictions']}")
        
        if self.live_stats['total_predictions'] > 0:
            success_rate = (self.live_stats['successful_predictions'] / self.live_stats['total_predictions']) * 100
            confidence_rate = (self.live_stats['confident_predictions'] / self.live_stats['total_predictions']) * 100
            print(f"📊 Success rate: {success_rate:.1f}%")
            print(f"🎯 Confidence rate: {confidence_rate:.1f}%")
        
        # Character analysis
        if self.live_stats['characters_this_session']:
            print(f"\\n🔤 CHARACTER DETECTION SUMMARY:")
            print(f"   📝 Unique characters: {len(self.live_stats['characters_this_session'])}")
            
            sorted_chars = sorted(self.live_stats['characters_this_session'].items(), 
                                key=lambda x: x[1], reverse=True)
            print("   🏆 Most detected:")
            for i, (char, count) in enumerate(sorted_chars[:3]):
                percentage = (count / self.live_stats['successful_predictions']) * 100
                print(f"      {i+1}. '{char}': {count} times ({percentage:.1f}%)")
        
        # Performance summary
        if self.performance_metrics['prediction_times']:
            avg_time = np.mean(list(self.performance_metrics['prediction_times']))
            min_time = np.min(list(self.performance_metrics['prediction_times']))
            max_time = np.max(list(self.performance_metrics['prediction_times']))
            print(f"\\n⚡ PERFORMANCE ANALYSIS:")
            print(f"   🔄 Avg prediction time: {avg_time:.3f}s")
            print(f"   ⚡ Fastest prediction: {min_time:.3f}s")
            print(f"   🐌 Slowest prediction: {max_time:.3f}s")
    
    def stop_prediction(self):
        """Stop enhanced prediction"""
        if self.is_running:
            self.is_running = False
            print("🛑 Stopping enhanced prediction...")
            if self.thread:
                self.thread.join(timeout=3)
            print("✅ Enhanced prediction stopped")
        else:
            print("ℹ️ Enhanced prediction not running")
    
    def get_prediction_data(self):
        """Get all collected prediction data"""
        return {
            'predictions': list(self.prediction_history),
            'confidences': list(self.confidence_history),
            'sensor_data': list(self.sensor_data_history),
            'timestamps': list(self.timestamp_history),
            'live_stats': self.live_stats.copy(),
            'performance_metrics': {k: list(v) for k, v in self.performance_metrics.items()}
        }

# Initialize Enhanced Continuous Predictor
if bengali_predictor is not None:
    print("🚀 INITIALIZING ENHANCED CONTINUOUS PREDICTOR...")
    print("=" * 55)
    
    enhanced_predictor = EnhancedContinuousPredictor(
        bengali_predictor=bengali_predictor,
        polling_interval=0.2  # 5 Hz
    )
    
    print("🎉 Enhanced continuous predictor ready!")
    
else:
    print("❌ Cannot initialize enhanced predictor")
    enhanced_predictor = None

🚀 INITIALIZING ENHANCED CONTINUOUS PREDICTOR...
📈 EnhancedContinuousPredictor initialized
   ⚡ Polling rate: 5.0 Hz
   💾 History buffer: 200 samples
🎉 Enhanced continuous predictor ready!


In [46]:
# 📊 Data Visualization and Analysis Tools
class PredictionDataAnalyzer:
    """
    Advanced data analysis and visualization for prediction results
    """
    
    def __init__(self, enhanced_predictor):
        """Initialize analyzer with predictor data"""
        self.enhanced_predictor = enhanced_predictor
        print("📊 PredictionDataAnalyzer initialized")
    
    def plot_real_time_confidence(self, last_n_predictions=50):
        """Plot confidence levels over time"""
        data = self.enhanced_predictor.get_prediction_data()
        
        if not data['confidences']:
            print("❌ No confidence data available")
            return
        
        confidences = data['confidences'][-last_n_predictions:]
        timestamps = data['timestamps'][-last_n_predictions:]
        
        if timestamps:
            # Convert to relative time (seconds from start)
            start_time = timestamps[0]
            relative_times = [(t - start_time) for t in timestamps]
            
            plt.figure(figsize=(12, 6))
            plt.plot(relative_times, confidences, 'b-', linewidth=2, marker='o', markersize=4)
            plt.axhline(y=0.8, color='g', linestyle='--', label='High Confidence (80%)')
            plt.axhline(y=0.6, color='orange', linestyle='--', label='Medium Confidence (60%)')
            plt.axhline(y=0.4, color='r', linestyle='--', label='Low Confidence (40%)')
            
            plt.xlabel('Time (seconds)')
            plt.ylabel('Confidence Level')
            plt.title(f'Real-Time Prediction Confidence (Last {len(confidences)} predictions)')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.ylim(0, 1)
            
            # Add statistics
            avg_conf = np.mean(confidences)
            plt.text(0.02, 0.98, f'Avg: {avg_conf:.1%}', transform=plt.gca().transAxes, 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            
            plt.tight_layout()
            plt.show()
        
        return confidences, relative_times
    
    def plot_character_frequency(self):
        """Plot character detection frequency"""
        stats = self.enhanced_predictor.live_stats
        char_data = stats.get('characters_this_session', {})
        
        if not char_data:
            print("❌ No character data available")
            return
        
        characters = list(char_data.keys())
        frequencies = list(char_data.values())
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(characters, frequencies, color='skyblue', edgecolor='darkblue')
        
        # Add value labels on bars
        for bar, freq in zip(bars, frequencies):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                    str(freq), ha='center', va='bottom', fontweight='bold')
        
        plt.xlabel('Bengali Characters')
        plt.ylabel('Detection Frequency')
        plt.title('Character Detection Frequency During Session')
        plt.grid(True, axis='y', alpha=0.3)
        
        # Add percentage annotations
        total = sum(frequencies)
        for i, (char, freq) in enumerate(char_data.items()):
            percentage = (freq / total) * 100
            plt.text(i, freq/2, f'{percentage:.1f}%', ha='center', va='center', 
                    color='white', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        return char_data
    
    def plot_prediction_timeline(self, last_n_predictions=100):
        """Plot prediction timeline with characters and confidence"""
        data = self.enhanced_predictor.get_prediction_data()
        
        if not data['predictions']:
            print("❌ No prediction data available")
            return
        
        predictions = data['predictions'][-last_n_predictions:]
        confidences = data['confidences'][-last_n_predictions:]
        timestamps = data['timestamps'][-last_n_predictions:]
        
        if not timestamps:
            return
        
        # Convert to relative time
        start_time = timestamps[0]
        relative_times = [(t - start_time) for t in timestamps]
        
        # Create figure with subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
        
        # Plot 1: Confidence over time
        ax1.plot(relative_times, confidences, 'b-', linewidth=2, marker='o', markersize=3)
        ax1.axhline(y=0.8, color='g', linestyle='--', alpha=0.7, label='High Confidence')
        ax1.set_ylabel('Confidence Level')
        ax1.set_title('Prediction Timeline: Confidence and Characters')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        ax1.set_ylim(0, 1)
        
        # Plot 2: Characters detected
        unique_chars = list(set(predictions))
        char_to_num = {char: i for i, char in enumerate(unique_chars)}
        char_numbers = [char_to_num[char] for char in predictions]
        
        scatter = ax2.scatter(relative_times, char_numbers, c=confidences, 
                            cmap='viridis', s=50, alpha=0.8)
        ax2.set_ylabel('Characters')
        ax2.set_xlabel('Time (seconds)')
        ax2.set_yticks(range(len(unique_chars)))
        ax2.set_yticklabels(unique_chars)
        ax2.grid(True, alpha=0.3)
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=ax2)
        cbar.set_label('Confidence Level')
        
        plt.tight_layout()
        plt.show()
        
        return predictions, confidences, relative_times
    
    def generate_session_report(self):
        """Generate comprehensive session report"""
        data = self.enhanced_predictor.get_prediction_data()
        stats = self.enhanced_predictor.live_stats
        
        if not data['predictions']:
            print("❌ No data available for report")
            return
        
        print("📋 COMPREHENSIVE SESSION REPORT")
        print("=" * 50)
        
        # Session overview
        total_time = stats.get('last_update_time', time.time()) - stats.get('start_time', time.time())
        print(f"⏰ Session Duration: {total_time:.1f} seconds")
        print(f"📊 Total Predictions: {len(data['predictions'])}")
        print(f"🚀 Average Rate: {len(data['predictions'])/total_time:.1f} predictions/sec")
        
        # Confidence analysis
        confidences = data['confidences']
        if confidences:
            print(f"\\n💪 CONFIDENCE ANALYSIS:")
            print(f"   📈 Average: {np.mean(confidences):.1%}")
            print(f"   📊 Median: {np.median(confidences):.1%}")
            print(f"   🔝 Maximum: {np.max(confidences):.1%}")
            print(f"   📉 Minimum: {np.min(confidences):.1%}")
            print(f"   📏 Std Dev: {np.std(confidences):.1%}")
            
            # Confidence distribution
            high_conf = sum(1 for c in confidences if c >= 0.8)
            med_conf = sum(1 for c in confidences if 0.6 <= c < 0.8)
            low_conf = sum(1 for c in confidences if c < 0.6)
            
            print(f"\\n🎯 CONFIDENCE DISTRIBUTION:")
            print(f"   🟢 High (≥80%): {high_conf} ({high_conf/len(confidences)*100:.1f}%)")
            print(f"   🟡 Medium (60-80%): {med_conf} ({med_conf/len(confidences)*100:.1f}%)")
            print(f"   🔴 Low (<60%): {low_conf} ({low_conf/len(confidences)*100:.1f}%)")
        
        # Character analysis
        char_data = stats.get('characters_this_session', {})
        if char_data:
            print(f"\\n🔤 CHARACTER ANALYSIS:")
            print(f"   📝 Unique Characters: {len(char_data)}")
            print(f"   🏆 Most Frequent: '{max(char_data.items(), key=lambda x: x[1])[0]}'")
            print(f"   📊 Character Distribution:")
            
            sorted_chars = sorted(char_data.items(), key=lambda x: x[1], reverse=True)
            for char, count in sorted_chars:
                percentage = (count / sum(char_data.values())) * 100
                print(f"      '{char}': {count} times ({percentage:.1f}%)")
        
        # Performance metrics
        perf_data = data['performance_metrics']
        if perf_data.get('prediction_times'):
            pred_times = perf_data['prediction_times']
            print(f"\\n⚡ PERFORMANCE METRICS:")
            print(f"   🔄 Avg Prediction Time: {np.mean(pred_times):.3f}s")
            print(f"   ⚡ Fastest: {np.min(pred_times):.3f}s")
            print(f"   🐌 Slowest: {np.max(pred_times):.3f}s")
        
        return data, stats

# Initialize Data Analyzer
if enhanced_predictor is not None:
    print("🚀 INITIALIZING PREDICTION DATA ANALYZER...")
    print("=" * 45)
    
    data_analyzer = PredictionDataAnalyzer(enhanced_predictor)
    
    print("🎉 Data analyzer ready!")
    
else:
    print("❌ Cannot initialize analyzer - enhanced predictor not available")
    data_analyzer = None

🚀 INITIALIZING PREDICTION DATA ANALYZER...
📊 PredictionDataAnalyzer initialized
🎉 Data analyzer ready!


In [32]:
# 🚀 Complete Continuous Prediction Demo System
def run_complete_continuous_demo(duration=30, confidence_threshold=0.75, 
                                show_live_dashboard=True, generate_plots=True):
    """
    Run a complete continuous prediction demo with all features
    
    Args:
        duration: Demo duration in seconds
        confidence_threshold: Minimum confidence for valid predictions
        show_live_dashboard: Show real-time dashboard updates
        generate_plots: Generate analysis plots after completion
    """
    
    if enhanced_predictor is None:
        print("❌ Enhanced predictor not available")
        return
    
    print("🚀 COMPLETE CONTINUOUS BENGALI PREDICTION DEMO")
    print("=" * 55)
    print(f"⏰ Duration: {duration} seconds")
    print(f"🎯 Confidence threshold: {confidence_threshold:.1%}")
    print(f"📊 Live dashboard: {'Enabled' if show_live_dashboard else 'Disabled'}")
    print(f"📈 Post-analysis: {'Enabled' if generate_plots else 'Disabled'}")
    print()
    print("📱 Instructions:")
    print("   • Make sure ESP32 glove is connected to Access Point")
    print("   • Perform Bengali sign gestures with your glove")
    print("   • Watch real-time predictions and statistics")
    print("   • Press Ctrl+C to stop early")
    print()
    
    # Check ESP32 connection
    print("🔍 Testing ESP32 connection...")
    if esp32_client.test_connection():
        print("✅ ESP32 connection successful!")
    else:
        print("⚠️ ESP32 connection failed - demo will continue but may show errors")
        print("💡 Make sure ESP32 is in Access Point mode at 192.168.4.1")
    
    print(f"\\n🎬 Starting {duration}s continuous prediction demo...")
    print("🛑 Press Ctrl+C to stop early")
    print("=" * 55)
    
    # Start enhanced prediction
    try:
        thread = enhanced_predictor.start_enhanced_prediction(
            duration_seconds=duration,
            confidence_threshold=confidence_threshold,
            show_live_stats=show_live_dashboard,
            update_interval=3.0  # Update dashboard every 3 seconds
        )
        
        # Wait for completion
        if thread:
            thread.join()
            
    except KeyboardInterrupt:
        print("\\n🛑 Demo stopped by user")
        enhanced_predictor.stop_prediction()
    
    # Generate analysis and plots
    if generate_plots and data_analyzer is not None:
        print("\\n📊 GENERATING POST-DEMO ANALYSIS...")
        print("=" * 40)
        
        try:
            # Generate comprehensive report
            data_analyzer.generate_session_report()
            
            # Create visualizations
            print("\\n📈 Creating visualizations...")
            
            # 1. Confidence timeline
            print("📊 Plotting confidence timeline...")
            data_analyzer.plot_real_time_confidence(last_n_predictions=100)
            
            # 2. Character frequency
            print("📊 Plotting character frequency...")
            data_analyzer.plot_character_frequency()
            
            # 3. Prediction timeline
            print("📊 Plotting prediction timeline...")
            data_analyzer.plot_prediction_timeline(last_n_predictions=50)
            
        except Exception as e:
            print(f"⚠️ Error generating plots: {e}")
    
    # Show final system statistics
    print("\\n📋 FINAL SYSTEM STATUS:")
    print("=" * 30)
    
    if esp32_client:
        esp32_client.print_stats()
    
    if bengali_predictor:
        bengali_predictor.print_stats()
    
    print("\\n🎉 Demo completed successfully!")
    
    return enhanced_predictor.get_prediction_data()

# 🔧 Quick Test Functions
def quick_prediction_test():
    """Quick single prediction test"""
    print("🔍 QUICK PREDICTION TEST")
    print("=" * 25)
    
    if bengali_predictor is None:
        print("❌ Predictor not available")
        return
    
    result = test_single_prediction(show_details=True)
    return result

def quick_continuous_test(duration=10):
    """Quick continuous prediction test"""
    print(f"⚡ QUICK {duration}S CONTINUOUS TEST")
    print("=" * 30)
    
    if enhanced_predictor is None:
        print("❌ Enhanced predictor not available")
        return
    
    thread = enhanced_predictor.start_enhanced_prediction(
        duration_seconds=duration,
        confidence_threshold=0.7,
        show_live_stats=True,
        update_interval=2.0
    )
    
    if thread:
        try:
            thread.join()
        except KeyboardInterrupt:
            enhanced_predictor.stop_prediction()
    
    return enhanced_predictor.get_prediction_data()

# 📊 System Status Check
def check_system_status():
    """Check status of all system components"""
    print("🔍 SYSTEM STATUS CHECK")
    print("=" * 25)
    
    # Check model
    model_status = "✅ Ready" if lstm_model is not None else "❌ Not loaded"
    print(f"🤖 LSTM Model: {model_status}")
    
    # Check preprocessing
    preprocessing_status = "✅ Ready" if (scaler is not None and label_encoder is not None) else "❌ Not loaded"
    print(f"⚙️ Preprocessing: {preprocessing_status}")
    
    # Check ESP32 client
    esp32_status = "✅ Initialized" if esp32_client is not None else "❌ Not initialized"
    print(f"📡 ESP32 Client: {esp32_status}")
    
    # Check predictor
    predictor_status = "✅ Ready" if bengali_predictor is not None else "❌ Not initialized"
    print(f"🧠 Bengali Predictor: {predictor_status}")
    
    # Check enhanced predictor
    enhanced_status = "✅ Ready" if enhanced_predictor is not None else "❌ Not initialized"
    print(f"📈 Enhanced Predictor: {enhanced_status}")
    
    # Check analyzer
    analyzer_status = "✅ Ready" if data_analyzer is not None else "❌ Not initialized"
    print(f"📊 Data Analyzer: {analyzer_status}")
    
    # Test ESP32 connection
    print("\\n🔍 Testing ESP32 connection...")
    if esp32_client and esp32_client.test_connection():
        print("✅ ESP32 Access Point: Connected")
    else:
        print("❌ ESP32 Access Point: Not connected")
    
    print("\\n🎯 SYSTEM READY FOR CONTINUOUS PREDICTION!")

print("🎉 CONTINUOUS DATA PREDICTION SYSTEM READY!")
print("=" * 50)
print("💡 Available Functions:")
print()
print("🚀 Main Functions:")
print("   • run_complete_continuous_demo(30) - Full 30s demo with analysis")
print("   • quick_continuous_test(10) - Quick 10s test")
print("   • quick_prediction_test() - Single prediction test")
print()
print("📊 Analysis Functions:")
print("   • data_analyzer.plot_real_time_confidence() - Confidence graph")
print("   • data_analyzer.plot_character_frequency() - Character distribution") 
print("   • data_analyzer.generate_session_report() - Detailed report")
print()
print("🔧 System Functions:")
print("   • check_system_status() - Check all components")
print("   • esp32_client.print_stats() - Connection statistics")
print("   • bengali_predictor.print_stats() - Prediction statistics")
print()
print("🚀 QUICK START: run_complete_continuous_demo(30)")

🎉 CONTINUOUS DATA PREDICTION SYSTEM READY!
💡 Available Functions:

🚀 Main Functions:
   • run_complete_continuous_demo(30) - Full 30s demo with analysis
   • quick_continuous_test(10) - Quick 10s test
   • quick_prediction_test() - Single prediction test

📊 Analysis Functions:
   • data_analyzer.plot_real_time_confidence() - Confidence graph
   • data_analyzer.plot_character_frequency() - Character distribution
   • data_analyzer.generate_session_report() - Detailed report

🔧 System Functions:
   • check_system_status() - Check all components
   • esp32_client.print_stats() - Connection statistics
   • bengali_predictor.print_stats() - Prediction statistics

🚀 QUICK START: run_complete_continuous_demo(30)


In [47]:
# 🔍 Check System Status
check_system_status()

🔍 SYSTEM STATUS CHECK
🤖 LSTM Model: ✅ Ready
⚙️ Preprocessing: ✅ Ready
📡 ESP32 Client: ✅ Initialized
🧠 Bengali Predictor: ✅ Ready
📈 Enhanced Predictor: ✅ Ready
📊 Data Analyzer: ✅ Ready
\n🔍 Testing ESP32 connection...
🔍 Testing connection to ESP32 at http://192.168.4.1:80...
⚠️ ESP32 responded with status: 404
❌ ESP32 Access Point: Not connected
\n🎯 SYSTEM READY FOR CONTINUOUS PREDICTION!


In [48]:
# 🔍 Diagnose Prediction Issues - Let's test step by step
def diagnose_prediction_issue():
    """Comprehensive diagnosis of why predictions aren't working"""
    
    print("🔍 DIAGNOSING PREDICTION ISSUES...")
    print("=" * 45)
    
    # Step 1: Test ESP32 connection with current IP
    print("📡 Step 1: Testing ESP32 connection at 192.168.4.1...")
    esp32_client.esp32_ip = "192.168.4.1"
    esp32_client.base_url = f"http://192.168.4.1:{esp32_client.port}"
    
    connection_test = esp32_client.test_connection()
    if connection_test:
        print("✅ ESP32 connection successful!")
    else:
        print("❌ Connection failed, but let's try fetching data directly...")
    
    # Step 2: Test data fetching
    print("\n📊 Step 2: Testing sensor data fetching...")
    sensor_data = esp32_client.get_sensor_data("/")  # Try root endpoint
    
    if sensor_data is None:
        print("❌ No data from root endpoint, trying /data...")
        sensor_data = esp32_client.get_sensor_data("/data")
    
    if sensor_data is None:
        print("❌ No data from /data endpoint, trying direct request...")
        try:
            import requests
            response = requests.get("http://192.168.4.1", timeout=5)
            print(f"📡 Direct response status: {response.status_code}")
            print(f"📄 Response content: {response.text[:200]}...")
            
            # Try to parse the CSV data manually
            csv_text = response.text.strip()
            if csv_text:
                print("🔧 Attempting manual CSV parsing...")
                lines = [line.strip() for line in csv_text.split('\n') if line.strip()]
                if lines:
                    data_line = lines[-1]
                    print(f"📊 Latest data line: {data_line}")
                    
                    try:
                        values = [float(x.strip()) for x in data_line.split(',')]
                        print(f"✅ Parsed {len(values)} sensor values: {values}")
                        
                        # Create sensor data manually
                        sensor_data = {
                            'timestamp': time.time(),
                            'sensor_data': values,
                            'sensor_count': len(values)
                        }
                        print("✅ Manual sensor data created successfully!")
                        
                    except Exception as e:
                        print(f"❌ Error parsing CSV: {e}")
                        return
            
        except Exception as e:
            print(f"❌ Direct request failed: {e}")
            return
    
    if sensor_data:
        print(f"✅ Sensor data received: {sensor_data}")
    else:
        print("❌ No sensor data available - cannot proceed with prediction")
        return
    
    # Step 3: Test model input requirements
    print(f"\n🧠 Step 3: Checking model requirements...")
    print(f"   📊 Model input shape: {bengali_predictor.input_shape}")
    print(f"   📏 Sequence length needed: {bengali_predictor.sequence_length}")
    print(f"   🎯 Features per step needed: {bengali_predictor.feature_count}")
    print(f"   📡 Sensor values available: {len(sensor_data['sensor_data'])}")
    
    if len(sensor_data['sensor_data']) < bengali_predictor.feature_count:
        print(f"⚠️ WARNING: Need {bengali_predictor.feature_count} features, got {len(sensor_data['sensor_data'])}")
        print("🔧 Padding or truncating to match model requirements...")
        
        # Fix the sensor data to match model requirements
        sensor_values = sensor_data['sensor_data']
        if len(sensor_values) < bengali_predictor.feature_count:
            # Pad with zeros
            sensor_values.extend([0.0] * (bengali_predictor.feature_count - len(sensor_values)))
        else:
            # Truncate
            sensor_values = sensor_values[:bengali_predictor.feature_count]
        
        sensor_data['sensor_data'] = sensor_values
        print(f"✅ Adjusted sensor data: {len(sensor_data['sensor_data'])} values")
    
    # Step 4: Test manual prediction
    print(f"\n🎯 Step 4: Testing manual prediction...")
    
    # Clear the buffer first
    bengali_predictor.data_buffer.clear()
    print(f"🧹 Cleared data buffer")
    
    # Add sensor data to buffer
    features = sensor_data['sensor_data'][:bengali_predictor.feature_count]
    bengali_predictor.data_buffer.append(features)
    print(f"📊 Added data to buffer: {len(bengali_predictor.data_buffer)}/{bengali_predictor.sequence_length}")
    
    # Since sequence_length is 1, we should be able to predict immediately
    if len(bengali_predictor.data_buffer) >= bengali_predictor.sequence_length:
        try:
            print("🔄 Attempting prediction...")
            
            # Prepare data
            sequence_data = np.array(list(bengali_predictor.data_buffer))
            print(f"📊 Sequence data shape: {sequence_data.shape}")
            
            sequence_data_scaled = bengali_predictor.scaler.transform(sequence_data)
            print(f"📊 Scaled data shape: {sequence_data_scaled.shape}")
            
            prediction_input = sequence_data_scaled.reshape(1, bengali_predictor.sequence_length, bengali_predictor.feature_count)
            print(f"📊 Prediction input shape: {prediction_input.shape}")
            
            # Make prediction
            prediction_probs = bengali_predictor.lstm_model.predict(prediction_input, verbose=0)
            predicted_class_idx = np.argmax(prediction_probs[0])
            confidence = float(prediction_probs[0][predicted_class_idx])
            
            # Decode character
            predicted_character = bengali_predictor.label_encoder.inverse_transform([predicted_class_idx])[0]
            
            print(f"🎉 PREDICTION SUCCESSFUL!")
            print(f"   🔤 Character: '{predicted_character}'")
            print(f"   💪 Confidence: {confidence:.1%}")
            print(f"   📊 All probabilities: {prediction_probs[0]}")
            
            return True
            
        except Exception as e:
            print(f"❌ Prediction failed: {e}")
            import traceback
            traceback.print_exc()
            return False
    else:
        print(f"❌ Buffer not ready: {len(bengali_predictor.data_buffer)}/{bengali_predictor.sequence_length}")
        return False

# Run the diagnosis
diagnose_prediction_issue()

🔍 DIAGNOSING PREDICTION ISSUES...
📡 Step 1: Testing ESP32 connection at 192.168.4.1...
🔍 Testing connection to ESP32 at http://192.168.4.1:80...
❌ Connection error: Cannot reach ESP32
❌ Connection failed, but let's try fetching data directly...

📊 Step 2: Testing sensor data fetching...
❌ No data from root endpoint, trying /data...
❌ No data from /data endpoint, trying direct request...
❌ Direct request failed: HTTPConnectionPool(host='192.168.4.1', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x722cf7a57ad0>, 'Connection to 192.168.4.1 timed out. (connect timeout=5)'))
